In [1]:
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)

TensorFlow version: 1.12.0


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from datetime import datetime


In [4]:
pc_df = pd.read_csv("PatientCorePopulatedTable.csv",na_values = ['Unknown'], infer_datetime_format = True, parse_dates = ['PatientDateOfBirth'])

In [5]:
# # pc_df = pc_df.drop(['PatientDateOfBirth'], axis = 1)
# pc_df = pc_df.drop(['PatientID'], axis = 1)
# pc_df["PatientAge"] = (pd.Timestamp(datetime.now()) - pc_df["PatientDateOfBirth"]).astype('<m8[Y]')

In [6]:
print (list(pc_df))
pc_df.shape

['PatientID', 'PatientGender', 'PatientDateOfBirth', 'PatientRace', 'PatientMaritalStatus', 'PatientLanguage', 'PatientPopulationPercentageBelowPoverty']


(10000, 7)

In [7]:
pc_df.loc[pc_df['PatientGender'] == 'female', 'PatientGender'] = 'Female'
pc_df.loc[pc_df['PatientRace'] == 'Aslan', 'PatientRace'] = 'Asian'
pc_df.loc[pc_df['PatientLanguage'] == 'Engrish', 'PatientLanguage'] = 'English'
pc_df.loc[pc_df['PatientPopulationPercentageBelowPoverty'] == 'text', 'PatientPopulationPercentageBelowPoverty'] = ''
# pc_df.loc[pc_df['PatientAge'] == '-17.000000', 'PatientAge'] = '17.000000'
s = pc_df.PatientPopulationPercentageBelowPoverty
pd.to_numeric(s, errors='coerce')
s.describe()
# pc_df['PatientPopulationPercentageBelowPoverty'] = pc_df['PatientPopulationPercentageBelowPoverty'].astype('category')

count     10000
unique     2410
top       19.33
freq         18
Name: PatientPopulationPercentageBelowPoverty, dtype: object

In [8]:
# pc_df = pc_df.drop(['PatientDateOfBirth'], axis = 1)
# pc_df = pc_df.drop(['PatientID'], axis = 1)
pc_df["PatientAge"] = (pd.Timestamp(datetime.now()) - pc_df["PatientDateOfBirth"]).astype('<m8[Y]')

In [9]:
pc_df.sample(5)


,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,PatientLanguage,PatientPopulationPercentageBelowPoverty,PatientAge
9214,5B8774C3-EC72-4430-85A7-2D2D14BC25FF,Male,1963-04-28 07:29:29.887,NaN,Single,Spanish,11.08,55.0
8895,9E1CFE90-3A76-4C71-A0A3-97DE7EEF1DC6,Female,1969-07-24 13:26:48.377,NaN,Married,Spanish,15.46,49.0
2095,3962BCE9-F512-492C-8DFF-B5653E74C13E,Male,1964-02-26 01:05:51.280,White,Married,Spanish,13.32,54.0
8579,488E6DC9-5300-4974-9265-6F65F70D88B2,Female,1955-05-20 07:31:28.027,African American,NaN,English,15.02,63.0
2712,3755E978-DE3E-484A-AA52-0F229512AD02,Male,1955-07-24 15:17:25.113,White,Single,Icelandic,15.14,63.0


In [10]:
def summerize_data(pc_df):
    for column in pc_df.columns:
        print (column)
        if pc_df.dtypes[column] == np.object: # Categorical data
            print (pc_df[column].value_counts())
        else:
            print (pc_df[column].describe()) 
        
        print ('\n')
    
summerize_data(pc_df)

PatientID
251BFC0B-B523-4683-94B7-EF0BFE7A263A    1
2E888F81-6E1B-48A2-B782-3092409A40F7    1
E6DD3357-7A3F-4294-9CE2-2B959425D457    1
6729E086-F7C5-4B48-9F83-160782CAE6CA    1
105C64F5-5AC6-468B-8D40-E02FC111A77D    1
19F78388-E150-4DC6-B87B-F4F070596F5D    1
E19BA22B-2480-4632-8935-53C62A707D68    1
F292FEFD-DF15-4D1D-8880-7E1D80C3936F    1
4AB007D8-EB13-49BF-9FB0-EF87EFB32131    1
707D9457-714B-42F5-BCFE-AEFE7316EC3E    1
AA4FE124-CBE0-4E69-9E5E-E2CAB4079A24    1
CDE992BB-BC9E-4438-8CF8-425D232877B8    1
F7D1A334-AD87-475B-84DE-22A694328021    1
D5B86C30-CE6C-45A4-ABBB-78DD5500EFB2    1
E021E8FA-2EC9-4A55-B0DE-5C2340642F32    1
DD237F3D-BB16-486E-8D2A-48BAA1FD7C21    1
4BAA1B65-4771-4FBA-A224-BA285B0F1E1B    1
0FF52150-EB65-484B-A975-BDA04F2702DF    1
F3769631-16A5-40A7-ADA5-97853F4F6445    1
BF184EA2-2E35-4631-BBF4-049DAE47F529    1
BCA9F5D4-04B6-454B-9E87-B30943DB6A7D    1
B4BE9BAA-DACA-46BF-96F2-D9B7165A231A    1
849023B7-F886-4130-AC16-576BE77612D0    1
9E0441B8-A33D-42F6-82F1-

In [11]:
# Encode the categorical features as numbers
def number_encode_features(pc_df):
    result = pc_df.copy()
    encoders = {}
    for column in result.columns:
        if result.dtypes[column] == np.object:
            encoders[column] = preprocessing.LabelEncoder()
            result[column] = encoders[column].fit_transform(result[column])
    return result, encoders


# Calculate the correlation and plot it
encoded_data, _ = number_encode_features(pc_df)
sns.heatmap(encoded_data.corr(), square=True)
plt.show()

encoded_data.tail(5)

TypeError: '<' not supported between instances of 'str' and 'float'

In [ ]:
pc_df.dtypes

In [ ]:
pc_df.to_csv('cleaned.csv')